In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# RQ2: In which phase of the technology life cycle path are the technologies?
# - Interest in topic: #Papers on a topic over time
# - Parallel dazu: Interest in topic: Number of citations to papers on a topic over time
#- Publikationstyp: Workshop -> Conference -> Journal 

In [ ]:
data = pd.read_csv('./data/500st.csv')
data.head()

In [ ]:
def Count(column,count):
    cnt = Counter()
    for item in data[column]: 
        
        cnt[item] += 1
    return cnt.most_common(count)

#_similarity = cosine_similarity()

In [ ]:
# - Parallel dazu: Interest in topic: Number of citations to papers on a topic over time

df = data[['Authors','Cites']].groupby('Authors').sum()


d = df['Cites'].loc[df['Cites'] > 400]
d.plot.bar()
#print(df['Cites'])


result = data.pivot_table(index='Title', 
                        columns='Year', 
                        values='Cites',
                        aggfunc=np.sum,
                        fill_value = 0)
#result

In [ ]:
#- Publikationstyp: Workshop -> Conference -> Journal 




In [ ]:
result